# MS-E2121 Exercise session 7
### Problem 7.1: Dantzig-Wolfe decomposition

Consider the following linear programming problem:

$$\begin{matrix}
\text{min} & &-x_{12} & & &-x_{22} &-x_{23}& \\
\text{s.t.} &x_{11} &+x_{12} &+x_{13} & & & &= 20 \\
& & & &x_{21} &+x_{22} &+x_{23} &= 20 \\
&-x_{11} & & &-x_{21} & & &= -20 \\
& &-x_{12} & & &-x_{22} & &= -10 \\
& & &-x_{13} & & &-x_{23} & = -10 \\
&x_{11} & & & & &+x_{23} &\le 15 \\
&x_{11,} &x_{12}, &x_{13}, &x_{21}, &x_{22}, &x_{23} &\ge 0\\
\end{matrix}$$

We wish to solve this problem using Dantzig-Wolfe decomposition, where the constraint $x_{11}+x_{23} \le 15$ is the only "coupling" constraint and the remaining constraints define a single subproblem.

The problem is then formulated as 

$$\begin{align*}
\text{min} & \sum_{k = 1}^K c_k ^\top x_k 	\\
\text{s.t.} & \sum_{k=1}^K A_k x_k = b \\
& x_k \in P_k, \ \forall k \in \{1,\dots,K\}. 
\end{align*}$$

with $P_k = \{x_k \geq 0 : D_kx_k = d_k\}$ and K=1. The constraint $x \in P = \{x \geq 0 : Dx = d\} $ defines our single subproblem.

**a)**

Consider the following two extreme points for the subproblem:

$$x^1 = (20,0,0,0,10,10),$$

and

$$x^2 = (0,10,10,20,0,0).$$

Construct a main problem in which $x$ is constrained to be a convex combination of $x^1$ and $x^2$. Find the optimal primal and dual solutions for the main problem.

**b)**

Using the dual variables calculated in part a), formulate the subproblem and find its optimal solution.

**c)**

What is the reduced cost of the variable $\lambda_3$ associated with the extreme point $x^3$ obtained from solving the subproblem in part b)?

**d)**

Compute a lower bound on the optimal cost.

In [1]:
using JuMP, GLPK

In [2]:
# Subproblem parameters
D = [1 1 1 0 0 0; 0 0 0 1 1 1; -1 0 0 -1 0 0; 0 -1 0 0 -1 0; 0 0 -1 0 0 -1] 
d = [20 20 -20 -10 -10]

# Main problem parameters
c = [0 -1 0 0 -1 -1]
A = [1 0 0 0 0 1]
b = 15

# The list of extreme points found so far
extremepoints = [[20.0 0 0 0 10 10], [0 10 10 20 0 0]];

I = 1:length(c) # Number of variables
M = 1:size(D)[1]; # Number of subproblem constraints 

We start by formulating the main problem, which is done by finding a convex combination of extreme points plus a nonnegative combination of extreme rays that satisfy the main problem constraints and minimize the objective function. Note that we don't have extreme rays at this point and we only have a single subproblem so we're only looking at a convex combination $x = \sum_{j \in J} \lambda^j x^j$ minimizing $c^\top x$ and satisfying our complicating constraint $Ax \le b$. 

In [3]:

J = 1:length(extremepoints)

mainproblem = Model(GLPK.Optimizer)
@variable(mainproblem, λ[J]>=0) # Weights for the extreme points (note that we have no extreme rays at this point)

# x_conv = sum(lambda[j]*x[j]), also feasible for the subproblem as x[j] are feasible points
@expression(mainproblem, x_conv[i in I], sum(λ[j]*extremepoints[j][i] for j in J)) 

@objective(mainproblem, Min, sum(c[i]*x_conv[i] for i in I))
@constraint(mainproblem, con, sum(A[i]*x_conv[i] for i in I) <= b)
@constraint(mainproblem, sumcon, sum(λ[j] for j in J) == 1)

println(mainproblem)
optimize!(mainproblem)

main_opt = objective_value(mainproblem)
λ_opt = value.(λ)
q_opt = dual.(con)
r_opt = dual(sumcon);

Min -10 λ[2] - 20 λ[1]
Subject to
 sumcon : λ[1] + λ[2] == 1.0
 con : 30 λ[1] <= 15.0
 λ[1] >= 0.0
 λ[2] >= 0.0



In [4]:
value.(x_conv)

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:6
And data, a 6-element Array{Float64,1}:
 10.0
  5.0
  5.0
 10.0
  5.0
  5.0

Since our extreme points $x^j$ are feasible for the subproblem, their convex combination is also feasible for the subproblem (because the set $P$ is convex). The main problem additionally constrains the combination to satisfy the complicating constraint $x_{11}+x_{23} \le 15$, and the optimal solution is thus primal feasible. It might not be optimal, however. In order to check optimality, we want to check the reduced costs. To do this efficiently without calculating all reduced costs, we solve the subproblem, also known as the *pricing problem*.

In our case, we can drop the subproblem index $k$ (we only have one) and the subproblem becomes

$$\begin{align*}
\text{min} \ & (c^\top - q^\top A) x \\
\text{s.t.} \ & x \in P. 
\end{align*}$$

In [5]:

subproblem = Model(GLPK.Optimizer)
@variable(subproblem, x[I] >= 0)
#%
@objective(subproblem, Min, sum((c[i] - sum(q_opt*A[:,i]))*x[i] for i in I))
@constraint(subproblem, [m in M], sum(D[m,i].*x[i] for i in I) == d[m])
#%
println(subproblem)
optimize!(subproblem)
sub_opt = objective_value(subproblem);

Min 0.3333333333333333 x[1] - x[2] - x[5] - 0.6666666666666667 x[6]
Subject to
 x[1] + x[2] + x[3] == 20.0
 x[4] + x[5] + x[6] == 20.0
 -x[1] - x[4] == -20.0
 -x[2] - x[5] == -10.0
 -x[3] - x[6] == -10.0
 x[1] >= 0.0
 x[2] >= 0.0
 x[3] >= 0.0
 x[4] >= 0.0
 x[5] >= 0.0
 x[6] >= 0.0



In [6]:
sub_opt

-13.333333333333336

The reduced cost of the variable $\lambda_3$ is calculated as $(c_k^\top - q^\top A_k)x_k^3 - r_k$, where $(c_k^\top - q^\top A_k)x_k^3$ is the optimal solution of the subproblem $k$. 

In [7]:
#%
c_r = sub_opt-r_opt
println("Reduced cost: $(c_r)")
#%

Reduced cost: -3.3333333333333357


The lower bound is obtained as $z + \sum_{k=1}^K (z_k-r_k)$, where $z$ is the optimal value for the main problem and $z_k$ the optimal value for subproblem $k$. Note that we only have one subproblem.

In [8]:
#%
lb = main_opt+c_r
println("The optimal solution is in the range [$(lb), $(main_opt)]")
#%

The optimal solution is in the range [-18.333333333333336, -15.0]


We can see that the solution we have is not yet optimal, so we can just add the new extreme point $x^3$ into our list and do another iteration of solving the main problem and subproblem (you can do this if you have time)

In [9]:
#%
x_opt = value.(x)
push!(extremepoints,x_opt')
println(x_opt)
#%

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:6
And data, a 6-element Array{Float64,1}:
 10.0
 10.0
  0.0
 10.0
  0.0
 10.0


In [10]:
#%
J = 1:length(extremepoints)
mainproblem = Model(GLPK.Optimizer)
@variable(mainproblem, λ[J]>=0)
@expression(mainproblem, x_conv[i in I], sum(λ[j]*extremepoints[j][i] for j in J))

@objective(mainproblem, Min, sum(c[i]*x_conv[i] for i in I))
@constraint(mainproblem, con, sum(A[i]*x_conv[i] for i in I) <= b)
@constraint(mainproblem, sumcon, sum(λ[j] for j in J) == 1)

println(mainproblem)
optimize!(mainproblem)
main_opt = objective_value(mainproblem)
λ_opt = value.(λ)
q_opt = dual.(con)
r_opt = dual(sumcon);
#%

Min -10 λ[2] - 20 λ[3] - 20 λ[1]
Subject to
 sumcon : λ[1] + λ[2] + λ[3] == 1.0
 con : 30 λ[1] + 20 λ[3] <= 15.0
 λ[1] >= 0.0
 λ[2] >= 0.0
 λ[3] >= 0.0



In [11]:
#%
subproblem = Model(GLPK.Optimizer)
@variable(subproblem, x[I] >= 0)
@objective(subproblem, Min, sum((c[i] - sum(q_opt*A[:,i]))*x[i] for i in I))
@constraint(subproblem, [m in M], sum(D[m,i].*x[i] for i in I) == d[m])

println(subproblem)
optimize!(subproblem)
sub_opt = objective_value(subproblem);
#%

Min 0.5 x[1] - x[2] - x[5] - 0.5 x[6]
Subject to
 x[1] + x[2] + x[3] == 20.0
 x[4] + x[5] + x[6] == 20.0
 -x[1] - x[4] == -20.0
 -x[2] - x[5] == -10.0
 -x[3] - x[6] == -10.0
 x[1] >= 0.0
 x[2] >= 0.0
 x[3] >= 0.0
 x[4] >= 0.0
 x[5] >= 0.0
 x[6] >= 0.0



In [12]:
#%
c_r = sub_opt-r_opt
println("Reduced cost: $(c_r)")
#%

Reduced cost: 0.0


In [13]:
#%
lb = main_opt+c_r
println("The optimal solution is in the range [$(lb), $(main_opt)]")
#%

The optimal solution is in the range [-17.5, -17.5]


In [14]:
#%
println("The optimal solution is $(main_opt)")
#%

The optimal solution is -17.5
